In [16]:
import os
import cv2

datadir_train = "classified/train"
datadir_test = "classified/test"
categories = ['1c', '2c', '5c', '10c', '20c', '50c', '1e', '2e']

training_data = []
for category in categories:
    path = os.path.join(datadir_train, category)
    label = categories.index(category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
        img_array = cv2.resize(img_array, (75, 75))
        training_data.append([img_array, label])

In [17]:
import numpy as np
x_train = []
y_train = []

for features, label in training_data:
    x_train.append(features)
    y_train.append(label)
    
x_train = np.array(x_train)

In [18]:
testing_data = []
for category in categories:
    path = os.path.join(datadir_test, category)
    class_num = categories.index(category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
        img_array = cv2.resize(img_array, (75, 75))
        testing_data.append([img_array, class_num])

In [19]:
x_test = []
y_test = []

for features, label in testing_data:
    x_test.append(features)
    y_test.append(label)
    
x_test = np.array(x_test)

In [131]:
distences = []
predict = x_test[75]
for image in training_data:
    euc = np.linalg.norm(image[0] - predict)
    distences.append([euc, image[1]])
distences.sort()

In [132]:
len(distences)

1291

In [129]:
len(x_train)

1291

In [130]:
len(distences)

1666681

In [133]:
from collections import Counter
def k_nearest_neighbors(predict, k):
    distences = []
    for image in training_data:
        distences.append([np.linalg.norm(image[0] - predict), image[1]])
    distences.sort()
    votes = [i[1] for i in distences[:k]]
    votes = ''.join(str(e) for e in votes)
    votes = votes.replace(',', '')
    votes = votes.replace(' ', '')
    result = Counter(votes).most_common(1)[0][0]
    return result

In [135]:
from time import time
start = time()
print(k_nearest_neighbors(x_test[72], 20))
end = time()
print(end-start)

6
0.040891408920288086


In [136]:
y_test = ''.join(str(e) for e in y_test)

In [150]:
start = time()
correct = 0
total = 0
prediction = []
for image in testing_data:
    prediction.append(k_nearest_neighbors(image[0], 1291))
    """if int(prediction) == image[1]:
        correct += 1
    total += 1
accuracy = correct/total"""
end = time()
print(end-start)

3.247319459915161


In [151]:
print(prediction)

['2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2']
